In [1]:
#|default_exp preprocessing

# PREPROCESSING

> API details.

In [2]:
#| export
import numpy as np
import h5py
import pandas as pd
from typing import Dict
from secrets import token_hex
from pathlib import Path
from scipy import stats
import pyfstat
import matplotlib.pyplot as plt
import os
import random
from joblib import Parallel, delayed
from tqdm import tqdm


22-11-07 16:52:25.863 pyfstat INFO    : Running PyFstat version 1.18.1
22-11-07 16:52:25.946 pyfstat.utils.importing INFO    : No $DISPLAY environment variable found, so importing matplotlib.pyplot with non-interactive 'Agg' backend.


In [3]:
#| export
def good_luck():
    return 'pass'

In [4]:
# | export
def read_file(filename):
    file_id = Path(filename).stem
    img = np.empty((2, 360, 128), dtype=np.float32)
    with h5py.File(filename, "r") as f:
        g = f[file_id]

        for ch, s in enumerate(["H1", "L1"]):
            a = g[s]["SFTs"][:, :4096] * 1e22  # Fourier coefficient complex64
            p = a.real**2 + a.imag**2  # power
            p /= np.mean(p)  # normalize
            p = np.mean(p.reshape(360, 128, 32), axis=2)  # compress 4096 -> 128
            img[ch] = p
    return img


def read_timestemp(filename):
    file_id = Path(filename).stem
    img = dict()
    with h5py.File(filename, "r") as f:
        img["L1"] = np.copy(f[file_id]["L1"]["timestamps_GPS"])
        img["H1"] = np.copy(f[file_id]["H1"]["timestamps_GPS"])
    return img


def get_random_name(len_k=16):
    token = token_hex(len_k)
    return token


def save_hdf(
    name: str,
    sft_h1: np.array,
    sft_l1: np.array,
    timestamps: Dict,
    frequency: np.array,
    meta_: dict,
):
    hf = h5py.File(f"{name}.h5", "w")
    g1 = hf.create_group(name.stem)
    h1 = g1.create_group("H1")
    h1.create_dataset("SFTs", data=sft_h1)
    h1.create_dataset("timestamps_GPS", data=timestamps["H1"])

    l1 = g1.create_group("L1")
    l1.create_dataset("SFTs", data=sft_l1)
    l1.create_dataset("timestamps_GPS", data=timestamps["L1"])

    hf.create_dataset("frequency_Hz", data=frequency)
    pd.DataFrame(meta_).to_csv(f"{name}.csv", index=False)


def constrained_sum_sample_pos(n, total):
    """Return a randomly chosen list of n positive integers summing to total.
    Each such list is equally likely to occur."""

    dividers = sorted(random.sample(range(1, total), n - 1))
    return [a - b for a, b in zip(dividers + [total], [0] + dividers)]

def get_random_sqrtx_noise():
    sqrtx_choice = [
        (2e-23, 2.5e-23),
        (1e-23, 1.5e-23),
        (3e-23, 3.5e-23),
        (4e-23, 4.5e-23),
        (5e-23, 5.5e-23),
        (6e-23, 7e-23),
    ]
    ch = random.choice(sqrtx_choice)
    return ch[0], ch[1]
    #return np.random.uniform(1e-23, 5e-23)

## DATA_V10

## DATA_V11

## DATA_V12

In [5]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()